# Importing packages

In [29]:
import os

import pandas as pd
import numpy as np

import cv2
from glob import glob as glob
from pathlib import Path

In [35]:
crack_prodir = Path("../data/processed/crack_temp")

In [36]:
def analyze_crack_intensity_v3(binary_img, area_thresh_low=0.07, area_thresh_high=0.10, width_thresh=30):

    img_height, img_width = binary_img.shape
    print(f"Image Size: {img_height} x {img_width}")
    img_area = img_height * img_width

    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    crack_features = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 10:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        bbox = binary_img[y:y+h, x:x+w]
        white_pixels = cv2.countNonZero(bbox)
        # bbox_area = w * h
        crack_ratio = white_pixels / img_area

        # Use minimum of width and height as "local width"
        local_width = min(w, h)

        # Combine area and width to decide intensity
        if crack_ratio < area_thresh_low or local_width < width_thresh:
            intensity = 0  # Low
        elif (crack_ratio > area_thresh_low and crack_ratio < area_thresh_high) or (local_width < width_thresh * 1.5):
            intensity = 1  # Medium
        else:
            intensity = 2  # High

        aspect_ratio = float(w) / h if h != 0 else 0

        print(f"Crack Ratio: {crack_ratio:.4f}, Local Width: {local_width}, Intensity: {intensity}")

        crack_features.append({
            'bbox': (x, y, w, h),
            'area': area,
            'white_pixels': white_pixels,
            'crack_ratio': crack_ratio,
            'aspect_ratio': aspect_ratio,
            'local_width': local_width,
            'intensity': intensity
        })

    return crack_features


In [37]:
def visualize_crack_intensity(img, crack_features):
    output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    for feat in crack_features:
        x, y, w, h = feat['bbox']
        intensity = feat['intensity']
        
        # Set color based on intensity level
        if intensity == 0:
            color = (0, 255, 0)      # Green for Low
        elif intensity == 1:
            color = (0, 255, 255)    # Yellow for Medium
        else:
            color = (0, 0, 255)      # Red for High

        cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
        cv2.putText(output, f"{intensity}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return output

In [39]:
# binary_img = your_preprocessed_crack_image
crack_images = list(crack_prodir.glob("*.jpg"))[80:90]

for img_path in crack_images:
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    
    binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
    features = analyze_crack_intensity_v3(binary)
    vis = visualize_crack_intensity(img, features)
    
    cv2.imshow(f"Crack Intensity - {img_path.name}", vis)
    cv2.waitKey(0)  # Wait for key press before moving to next image

cv2.destroyAllWindows()

Image Size: 227 x 227
Crack Ratio: 0.0860, Local Width: 122, Intensity: 1
Image Size: 227 x 227
Crack Ratio: 0.0883, Local Width: 154, Intensity: 1
Image Size: 227 x 227
Crack Ratio: 0.0902, Local Width: 103, Intensity: 1
Image Size: 227 x 227
Crack Ratio: 0.0051, Local Width: 19, Intensity: 0
Crack Ratio: 0.1194, Local Width: 150, Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.0867, Local Width: 156, Intensity: 1
Image Size: 227 x 227
Crack Ratio: 0.0436, Local Width: 96, Intensity: 0
Crack Ratio: 0.1140, Local Width: 81, Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.0849, Local Width: 142, Intensity: 1
Image Size: 227 x 227
Crack Ratio: 0.1105, Local Width: 141, Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.0884, Local Width: 52, Intensity: 1
Crack Ratio: 0.1351, Local Width: 115, Intensity: 2
Image Size: 227 x 227
Crack Ratio: 0.0009, Local Width: 8, Intensity: 0
Crack Ratio: 0.0624, Local Width: 149, Intensity: 0
